In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier

import seaborn as sns
sns.set(rc={'figure.figsize':(20, 20)})

In [3]:
train_df = pd.read_csv("../input/students-drop-out-prediction/train.csv")
test_df = pd.read_csv("../input/students-drop-out-prediction/test.csv")

In [4]:
all_col_names = list(set(train_df.columns) - set(["label", "id"])) 
binary_col_names = ["v_1", "v_26", "v_11", "v_14", "v_30", "v_28", "v_9", "v_27"]
non_binary_col_names = list(set(all_col_names) - set(binary_col_names))

In [5]:
transformer = ColumnTransformer(
    [
        ("drop_id", "drop", ["id"]),
        ("scale_non_binary", StandardScaler(), non_binary_col_names)
    ],
    remainder="passthrough"
)
transform_train_df = transformer.fit_transform(train_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_train_df = pd.DataFrame(data=transform_train_df, columns=feature_names)
transform_train_df = transform_train_df.astype(dict([(col, "int") for col in binary_col_names]))

In [6]:
transform_train_df1 = transform_train_df.copy()
y_train_com = transform_train_df1.pop("label")
X_train_com = transform_train_df1
X_train, X_valid, y_train, y_valid = train_test_split(X_train_com, y_train_com, test_size=0.3, random_state=1)

In [7]:
n_models = 20
models = []
for i in range(n_models):
    model = DecisionTreeClassifier(
        class_weight="balanced",
        random_state=i+1,
        ccp_alpha=0.001,
        max_depth=10,
        max_features=None,
        min_samples_leaf=10,
        min_samples_split=5
    )
    models.append((f"model{i+1}", model))
voting = VotingClassifier(models, voting="soft")
voting.fit(X_train, y_train)
preds = voting.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.70      0.62      0.66       375
         1.0       0.79      0.67      0.72       540
         2.0       0.35      0.54      0.42       224

    accuracy                           0.63      1139
   macro avg       0.61      0.61      0.60      1139
weighted avg       0.67      0.63      0.64      1139



In [8]:
bag_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(
        class_weight="balanced",
        max_features=None,
    ),
    random_state=1
)
bag_clf.fit(X_train, y_train)
preds = bag_clf.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.72      0.70      0.71       375
         1.0       0.74      0.90      0.81       540
         2.0       0.48      0.26      0.34       224

    accuracy                           0.71      1139
   macro avg       0.65      0.62      0.62      1139
weighted avg       0.68      0.71      0.68      1139



param_grid = {
    "n_estimators": [5,15, 20],
    "max_samples": [0.5, 0.75, 1],
    "max_features": [0.5, 0.75, 1],
}
model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(
        class_weight="balanced",
        max_features=None,
    ),
    random_state=1
)
search = GridSearchCV(model, param_grid, scoring="f1_macro", refit=True, cv=3)
search.fit(X_train_com, y_train_com)

search.best_params_

search.best_score_

param_grid = {
    "base_estimator__ccp_alpha": [0.0001, 0.001, 0.01],
    "base_estimator__max_depth": [5, 10, 15],
    "base_estimator__min_samples_split": [5, 10, 15],
    "base_estimator__min_samples_leaf": [5, 10, 15],
}
model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(
        class_weight="balanced",
        max_features=None,
    ),
    random_state=1,
    max_features=0.75,
    max_samples=0.5,
    n_estimators=20
)
search = GridSearchCV(model, param_grid, scoring="f1_macro", refit=True, cv=3)
search.fit(X_train_com, y_train_com)

search.best_params_

search.best_score_

In [9]:
model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(
        class_weight="balanced",
        max_features=None,
        ccp_alpha=0.0001,
        max_depth=10,
        min_samples_leaf=10,
        min_samples_split=5
    ),
    random_state=1,
    max_features=0.75,
    max_samples=0.5,
    n_estimators=20
)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.79      0.68      0.73       375
         1.0       0.79      0.86      0.82       540
         2.0       0.46      0.46      0.46       224

    accuracy                           0.72      1139
   macro avg       0.68      0.67      0.67      1139
weighted avg       0.72      0.72      0.72      1139



### Lets see if random forest can beat this!

In [10]:
# Good performance without tuning
model = RandomForestClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.78      0.71      0.74       375
         1.0       0.73      0.96      0.83       540
         2.0       0.63      0.25      0.35       224

    accuracy                           0.74      1139
   macro avg       0.71      0.64      0.64      1139
weighted avg       0.73      0.74      0.71      1139



param_grid = {
    "min_samples_split": [2, 5, 10],
    "max_samples": [0.5, 0.9],
    "n_estimators": [100, 200],
    "min_samples_leaf": [10, 20],
    "max_depth": [10, 20],
    "ccp_alpha": [0.0001, 0.001]
}
model = RandomForestClassifier(
    max_features=None,
    random_state=1,
    class_weight="balanced",
    min_samples_leaf=10,
    n_estimators=100
)
search = GridSearchCV(model, param_grid, scoring="f1_macro", refit=True, cv=3)
search.fit(X_train_com, y_train_com)

search.best_params_

search.best_score_

In [11]:
# Similar to BaggingClassifier
model = RandomForestClassifier(
    max_features=None,
    random_state=1,
    class_weight="balanced",
    n_estimators=100,
    max_samples=0.9,
    min_samples_split=2,
    min_samples_leaf=10,
    max_depth=10,
    ccp_alpha=0.0001
)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.79      0.65      0.71       375
         1.0       0.78      0.84      0.81       540
         2.0       0.46      0.52      0.49       224

    accuracy                           0.71      1139
   macro avg       0.68      0.67      0.67      1139
weighted avg       0.72      0.71      0.71      1139



In [12]:
model = RandomForestClassifier(
    max_features=None,
    random_state=1,
    class_weight="balanced",
    n_estimators=100,
    max_samples=0.9,
    min_samples_split=2,
    min_samples_leaf=10,
    max_depth=10,
    ccp_alpha=0.0001
)
model.fit(X_train_com, y_train_com)

RandomForestClassifier(ccp_alpha=0.0001, class_weight='balanced', max_depth=10,
                       max_features=None, max_samples=0.9, min_samples_leaf=10,
                       random_state=1)

In [13]:
# We do not need 'id' column
# Scale non binary features
test_df = pd.read_csv("../input/students-drop-out-prediction/test.csv")
transform_test_data = transformer.fit_transform(test_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_test_df = pd.DataFrame(data=transform_test_data, columns=feature_names)
transform_test_df = transform_test_df.astype(dict([(col, "int") for col in binary_col_names]))
preds = [int(i) for i in model.predict(transform_test_df)]
submission = pd.DataFrame({"id": test_df["id"].to_numpy(), "label": preds})
submission.to_csv("submission.csv", index=False)